# Deliverable 3. Create a Travel Itinerary Map.
----
1. Import the vacation CSV file. 
2. From the map above with pick 4 cities that are in close proximity (on the same continent) that the customer would travel to and create a directions layer map.
3. For the "travel_mode" use either "DRIVING", "BICYCLING", or "WALKING".
4. Take a screenshot of the route and save it. 
5. Then create a marker layer for the four cities. Each city will should have a pop-up marker that contains
    - The hotel name
    - The city
    - The country
    - The current weather description and the maximum temperature
---

## Make sure the initial dependencies and the Google API key are imported.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

In [2]:
# Configure gmaps
gmaps.configure(api_key=g_key)

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [3]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,6,Cabo San Lucas,MX,72.52,scattered clouds,22.8909,-109.9124,Hotel Tesoro
1,7,Kapaa,US,82.72,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
2,8,Rikitea,PF,79.61,clear sky,-23.1203,-134.9692,People ThankYou
3,17,Banda Aceh,ID,75.54,overcast clouds,5.5577,95.3222,OYO 884 Rumoh PMI Hotel
4,19,Atuona,PF,78.85,clear sky,-9.8000,-139.0333,Villa Enata


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [4]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## In this step, you will set-up the pop-up markers. Review the code to create a marker layer map of the vacation search results. This code is the same as in Deliverable 2.

In [5]:
# Review the formatting template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [6]:
# Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

## From the vacation search map, *choose four cities* that a customer might want to visit. They should be close together and in the same country.

In [7]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# The starting and ending city should be the same city.
vacation_start = vacation_df.loc[vacation_df['City'] == 'Rikitea']
vacation_end = vacation_df.loc[vacation_df['City'] == 'Rikitea']
vacation_stop1 = vacation_df.loc[vacation_df['City'] == 'Cabo San Lucas']
vacation_stop2 = vacation_df.loc[vacation_df['City'] == 'Atuona']
vacation_stop3 = vacation_df.loc[vacation_df['City'] == 'Banda Aceh']

## Use the `to_numpy()` function and list indexing to write code to retrieve the latitude-longitude pairs as tuples from each city DataFrame.

In [8]:
# Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = tuple(vacation_start[["Lat", "Lng"]].to_numpy()[0])
end = tuple(vacation_end[["Lat", "Lng"]].to_numpy()[0])
stop1 = tuple(vacation_stop1[["Lat", "Lng"]].to_numpy()[0])
stop2 = tuple(vacation_stop2[["Lat", "Lng"]].to_numpy()[0])
stop3 = tuple(vacation_stop3[["Lat", "Lng"]].to_numpy()[0])

## Use the [gmaps documentation](https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#directions-layer) to create a directions layer map using the variables from the previous step. Where the starting and ending city are the same city, the `waypoints` are the three other cities, and the `travel_mode` is either `"DRIVING"`, `"BICYCLING"`, or `"WALKING"`.

In [9]:
# Define a new figure object
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Create a direction layer map using the start and end latitude-longitude pairs, and stop1, stop2, and stop3 as the waypoints.
# The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
start = start
end = end
waypoints = [stop1, stop2, stop3]
directions = gmaps.directions_layer(start, end, waypoints=waypoints, travel_mode='WALKING')
directions = gmaps.directions_layer(start, end, waypoints=waypoints, travel_mode='DRIVING')
directions = gmaps.directions_layer(start, end, waypoints=waypoints, travel_mode='BICYCLING')



# Add the layer to the map
fig.add_layer(directions)

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))

## Use the provided `concat()` function code snippet to combine the four separate city DataFrames into one DataFrame.

In [10]:
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat(
    [
        vacation_start,
        vacation_stop1,
        vacation_stop2,
        vacation_stop3
    ],
    ignore_index = True
)

# Display sample data
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,8,Rikitea,PF,79.61,clear sky,-23.1203,-134.9692,People ThankYou
1,6,Cabo San Lucas,MX,72.52,scattered clouds,22.8909,-109.9124,Hotel Tesoro
2,19,Atuona,PF,78.85,clear sky,-9.8000,-139.0333,Villa Enata
3,17,Banda Aceh,ID,75.54,overcast clouds,5.5577,95.3222,OYO 884 Rumoh PMI Hotel


## Refactor the code from Step 6 to create a new marker layer map of the cities on the travel route.

In [11]:
# Review the formatting template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [12]:
# Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


# Define a new figure object
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Add the layer to the map
fig.add_layer(marker_layer)

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))